In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np; np.random.seed(0)

from extquadcontrol import ExtendedQuadratic, dp_infinite, dp_finite
from scipy.linalg import solve_discrete_are

# LQR

We verify that our implementation matches the controller returned by the infinite horizon Riccati Recursion.

In [3]:
n, m = 5,5
K = 1
N = 1
T = 25

As = np.random.randn(1,1,n,n)
Bs = np.random.randn(1,1,n,m)
cs = np.zeros((1,1,n))
gs = [ExtendedQuadratic(np.eye(n+m),np.zeros(n+m),0) for _ in range(K)]
Pi = np.eye(K)
def sample(t, N):
    A = np.zeros((N,K,n,n)); A[:] = As
    B = np.zeros((N,K,n,m)); B[:] = Bs
    c = np.zeros((N,K,n)); c[:] = cs
    g = [gs for _ in range(N)]
    return A,B,c,g,Pi
g_T = [ExtendedQuadratic(np.eye(n),np.zeros(n),0) for _ in range(K)]
Vs, Qs, policies = dp_finite(sample, g_T, T, N)

In [4]:
Vs[0][0].P

array([[ 2.46137441, -0.77024827, -0.33980347,  0.66651709, -0.69610513],
       [-0.77024827,  6.24869755,  1.74646601,  0.93560635,  2.00406741],
       [-0.33980347,  1.74646601,  1.98781933,  0.57979962,  1.51304143],
       [ 0.66651709,  0.93560635,  0.57979962,  2.14107031,  1.05984051],
       [-0.69610513,  2.00406741,  1.51304143,  1.05984051,  3.91844687]])

In [5]:
policies[0][0][0]

array([[-0.1531434 ,  1.48273977,  0.75353673,  0.66434926,  1.25074963],
       [-0.7184896 , -0.28190404, -0.01678578, -0.42554618,  0.20588423],
       [ 0.27487237,  0.15280488, -0.35387326,  0.21502023, -0.45298026],
       [-0.71000192,  0.38519929,  0.17111865, -0.1229067 ,  0.62503111],
       [-0.50136684,  0.86866064, -0.02695909, -0.51365859, -0.46328911]])

In [6]:
A = As[0,0]
B = Bs[0,0]
Q = np.eye(n)
R = np.eye(m)
def solve_finite_time():
    P = Q
    for _ in range(50):
        P = Q+A.T@P@A-A.T@P@B@np.linalg.solve(R+B.T@P@B,B.T@P@A)
    K = -np.linalg.solve(R+B.T@P@B,B.T@P@A)
    return P, K
P, K = solve_finite_time()

In [7]:
P

array([[ 2.46137441, -0.77024827, -0.33980347,  0.66651709, -0.69610513],
       [-0.77024827,  6.24869755,  1.74646601,  0.93560635,  2.00406741],
       [-0.33980347,  1.74646601,  1.98781933,  0.57979962,  1.51304143],
       [ 0.66651709,  0.93560635,  0.57979962,  2.14107031,  1.05984051],
       [-0.69610513,  2.00406741,  1.51304143,  1.05984051,  3.91844687]])

In [8]:
K

array([[-0.1531434 ,  1.48273977,  0.75353673,  0.66434926,  1.25074963],
       [-0.7184896 , -0.28190404, -0.01678578, -0.42554618,  0.20588423],
       [ 0.27487237,  0.15280488, -0.35387326,  0.21502023, -0.45298026],
       [-0.71000192,  0.38519929,  0.17111865, -0.1229067 ,  0.62503111],
       [-0.50136684,  0.86866064, -0.02695909, -0.51365859, -0.46328911]])

### Infinite-horizon

In [9]:
A = np.random.randn(1,1,n,n)
B = np.random.randn(1,1,n,m)
c = np.zeros((1,1,n))
g = [[ExtendedQuadratic(np.eye(n+m),np.zeros(n+m),0)]]
Pi = np.ones((1,1))
def sample(t):
    return A,B,c,g,Pi
V, Qs, policy = dp_infinite(sample, 50, 1)
V[0].P

array([[ 3.08277538e+00,  1.06380848e+00,  7.23507249e-01,
         3.42727302e+00, -2.63851038e-01],
       [ 1.06380848e+00,  2.24055073e+00,  1.04169334e-01,
         1.80452627e+00, -1.39624160e-01],
       [ 7.23507249e-01,  1.04169334e-01,  1.60752741e+00,
         6.61172514e-01, -7.55649163e-03],
       [ 3.42727302e+00,  1.80452627e+00,  6.61172514e-01,
         8.19011957e+00, -6.44350241e-01],
       [-2.63851038e-01, -1.39624160e-01, -7.55649163e-03,
        -6.44350241e-01,  1.10281654e+00]])

In [10]:
A = A[0,0]
B = B[0,0]

In [11]:
P = solve_discrete_are(A,B,Q,R)
P

array([[ 3.08277538e+00,  1.06380848e+00,  7.23507249e-01,
         3.42727302e+00, -2.63851038e-01],
       [ 1.06380848e+00,  2.24055073e+00,  1.04169334e-01,
         1.80452627e+00, -1.39624160e-01],
       [ 7.23507249e-01,  1.04169334e-01,  1.60752741e+00,
         6.61172514e-01, -7.55649163e-03],
       [ 3.42727302e+00,  1.80452627e+00,  6.61172514e-01,
         8.19011957e+00, -6.44350241e-01],
       [-2.63851038e-01, -1.39624160e-01, -7.55649163e-03,
        -6.44350241e-01,  1.10281654e+00]])